In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_folder = "/content/drive/MyDrive/work/Finalproject/housing_app_fall25-main"
%cd "{base_folder}"

/content/drive/MyDrive/work/Finalproject/housing_app_fall25-main


In [2]:
# =============================================================================
# 04 GENERATE STREAMLIT INPUT SCHEMA (CLASSIFICATION)
# =============================================================================

import sqlite3
import pandas as pd
import json
from pathlib import Path

# -------------------- PATHS --------------------
BASE_FOLDER = "/content/drive/MyDrive/work/Finalproject/housing_app_fall25-main"
DB_PATH = f"{BASE_FOLDER}/data/student_performance.db"
OUTPUT_PATH = f"{BASE_FOLDER}/data/data_schema.json"

print("="*80)
print("GENERATING STREAMLIT DATA SCHEMA")
print("="*80)

# -------------------- LOAD DATA --------------------
conn = sqlite3.connect(DB_PATH)

df = pd.read_sql_query("""
SELECT
    s.age,
    s.gender,
    s.internet_access,
    s.sleep_hours,
    s.sleep_quality,
    s.class_attendance,
    c.course_name AS course,
    c.study_method,
    c.facility_rating,
    e.exam_difficulty,
    e.study_hours,
    e.exam_score
FROM students s
JOIN exams e ON e.student_id = s.student_id
JOIN courses c ON c.course_id = e.course_id
""", conn)

conn.close()

print("Rows loaded:", len(df))
print("\nSample:")
display(df.head())

# -------------------- FEATURE GROUPS --------------------
numerical_features = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = df.select_dtypes(include=["object"]).columns.tolist()

schema = {
    "numerical": {},
    "categorical": {}
}

# -------------------- NUMERICAL STATS --------------------
print("\n" + "-"*80)
print("NUMERICAL FEATURES")
print("-"*80)

for col in numerical_features:
    schema["numerical"][col] = {
        "min": float(df[col].min()),
        "max": float(df[col].max()),
        "mean": float(df[col].mean()),
        "median": float(df[col].median())
    }

    print(
        f"{col:<20} "
        f"min={df[col].min():.2f} "
        f"max={df[col].max():.2f} "
        f"mean={df[col].mean():.2f} "
        f"median={df[col].median():.2f}"
    )

# -------------------- CATEGORICAL STATS --------------------
print("\n" + "-"*80)
print("CATEGORICAL FEATURES")
print("-"*80)

for col in categorical_features:
    values = df[col].value_counts()
    schema["categorical"][col] = {
        "unique_values": values.index.tolist(),
        "value_counts": values.to_dict()
    }

    print(f"\n{col}:")
    for k, v in values.items():
        print(f"  {k}: {v}")

# -------------------- SAVE JSON --------------------
Path(OUTPUT_PATH).parent.mkdir(parents=True, exist_ok=True)

with open(OUTPUT_PATH, "w") as f:
    json.dump(schema, f, indent=2)

print("\n" + "="*80)
print("✓ data_schema.json CREATED")
print("Path:", OUTPUT_PATH)
print("="*80)

# -------------------- PREVIEW --------------------
print("\nSCHEMA PREVIEW:")
print(json.dumps(schema, indent=2))


GENERATING STREAMLIT DATA SCHEMA
Rows loaded: 20000

Sample:


,age,gender,internet_access,sleep_hours,sleep_quality,class_attendance,course,study_method,facility_rating,exam_difficulty,study_hours,exam_score
0,17,male,yes,7.4,poor,92.9,diploma,coaching,low,hard,2.78,58.9
1,23,other,yes,4.6,average,64.8,bca,online videos,medium,moderate,3.37,54.8
2,22,male,yes,8.5,poor,76.8,b.sc,coaching,high,moderate,7.88,90.3
3,20,other,yes,5.8,average,48.4,diploma,online videos,low,moderate,0.67,29.7
4,20,female,yes,9.8,poor,71.6,diploma,coaching,low,moderate,0.89,43.7



--------------------------------------------------------------------------------
NUMERICAL FEATURES
--------------------------------------------------------------------------------
age                  min=17.00 max=24.00 mean=20.47 median=20.00
sleep_hours          min=4.10 max=9.90 mean=7.01 median=7.00
class_attendance     min=40.60 max=99.40 mean=70.02 median=69.90
study_hours          min=0.08 max=7.91 mean=4.01 median=4.04
exam_score           min=19.60 max=100.00 mean=62.51 median=62.60

--------------------------------------------------------------------------------
CATEGORICAL FEATURES
--------------------------------------------------------------------------------

gender:
  other: 6726
  male: 6695
  female: 6579

internet_access:
  yes: 16988
  no: 3012

sleep_quality:
  average: 6694
  poor: 6687
  good: 6619

course:
  bca: 2902
  ba: 2896
  b.sc: 2878
  b.com: 2864
  bba: 2836
  diploma: 2826
  b.tech: 2798

study_method:
  self-study: 4079
  online videos: 4069
  coach

In [ ]:
# =============================================================================
# ANALYZE STUDENT PERFORMANCE DATA FOR STREAMLIT APP
# - Numerical feature stats (min/max/mean/median)
# - Categorical feature unique values
# - Save schema as data_schema.json
# =============================================================================

import sqlite3
import pandas as pd
import json
from pathlib import Path

print("=" * 80)
print("ANALYZING STUDENT PERFORMANCE DATA FOR STREAMLIT APP")
print("=" * 80)

# -------------------------------------------------------------------------
# Paths
# -------------------------------------------------------------------------
BASE_FOLDER = "/content/drive/MyDrive/work/Finalproject/housing_app_fall25-main"
DB_PATH = f"{BASE_FOLDER}/data/student_performance.db"
OUTPUT_FILE = f"{BASE_FOLDER}/data/data_schema.json"

# -------------------------------------------------------------------------
# Load data from SQLite (CORRECT JOIN)
# -------------------------------------------------------------------------
conn = sqlite3.connect(DB_PATH)

df = pd.read_sql_query(
    """
    SELECT
        s.age,
        s.gender,
        s.internet_access,
        s.sleep_hours,
        s.sleep_quality,
        s.class_attendance,

        c.course_name      AS course,
        c.study_method,
        c.facility_rating,

        e.exam_difficulty,
        e.study_hours,
        e.exam_score,
        e.pass_fail
    FROM students s
    JOIN exams e
        ON e.student_id = s.student_id
    JOIN courses c
        ON c.course_id = e.course_id
    """,
    conn
)

conn.close()

print(f"Rows loaded: {len(df)}")
display(df.head())

# -------------------------------------------------------------------------
# Feature definitions (for Streamlit UI)
# -------------------------------------------------------------------------
numerical_features = [
    "age",
    "sleep_hours",
    "class_attendance",
    "study_hours",
    "exam_score",
]

categorical_features = [
    "gender",
    "internet_access",
    "sleep_quality",
    "course",
    "study_method",
    "facility_rating",
    "exam_difficulty",
]

# -------------------------------------------------------------------------
# Build schema
# -------------------------------------------------------------------------
data_schema = {
    "numerical": {},
    "categorical": {}
}

print("\n" + "-" * 80)
print("NUMERICAL FEATURES")
print("-" * 80)
print(f"{'Feature':<20} {'Min':<10} {'Max':<10} {'Mean':<10} {'Median':<10}")
print("-" * 80)

for feature in numerical_features:
    min_val = float(df[feature].min())
    max_val = float(df[feature].max())
    mean_val = float(df[feature].mean())
    median_val = float(df[feature].median())

    data_schema["numerical"][feature] = {
        "min": min_val,
        "max": max_val,
        "mean": mean_val,
        "median": median_val
    }

    print(
        f"{feature:<20} "
        f"{min_val:<10.2f} "
        f"{max_val:<10.2f} "
        f"{mean_val:<10.2f} "
        f"{median_val:<10.2f}"
    )

# -------------------------------------------------------------------------
# Categorical features
# -------------------------------------------------------------------------
print("\n" + "-" * 80)
print("CATEGORICAL FEATURES")
print("-" * 80)

for feature in categorical_features:
    unique_values = sorted(df[feature].dropna().unique().tolist())
    value_counts = df[feature].value_counts().to_dict()

    data_schema["categorical"][feature] = {
        "unique_values": unique_values,
        "value_counts": value_counts
    }

    print(f"\n{feature}:")
    print(f"  Unique values: {unique_values}")
    print("  Value counts:")
    for val, cnt in value_counts.items():
        pct = cnt / len(df) * 100
        print(f"    {val}: {cnt} ({pct:.1f}%)")

# -------------------------------------------------------------------------
# Save schema
# -------------------------------------------------------------------------
with open(OUTPUT_FILE, "w") as f:
    json.dump(data_schema, f, indent=2)

print("\n" + "=" * 80)
print(f"✓ Data schema saved to: {OUTPUT_FILE}")
print("=" * 80)

# Show JSON preview
print("\n" + "-" * 80)
print("GENERATED SCHEMA (data_schema.json)")
print("-" * 80)
print(json.dumps(data_schema, indent=2))

print("\n" + "=" * 80)
print("DONE! Use data_schema.json in your Streamlit app")
print("=" * 80)


ANALYZING STUDENT PERFORMANCE DATA FOR STREAMLIT APP
Rows loaded: 20000


,age,gender,internet_access,sleep_hours,sleep_quality,class_attendance,course,study_method,facility_rating,exam_difficulty,study_hours,exam_score,pass_fail
0,17,male,yes,7.4,poor,92.9,diploma,coaching,low,hard,2.78,58.9,pass
1,23,other,yes,4.6,average,64.8,bca,online videos,medium,moderate,3.37,54.8,pass
2,22,male,yes,8.5,poor,76.8,b.sc,coaching,high,moderate,7.88,90.3,pass
3,20,other,yes,5.8,average,48.4,diploma,online videos,low,moderate,0.67,29.7,fail
4,20,female,yes,9.8,poor,71.6,diploma,coaching,low,moderate,0.89,43.7,fail



--------------------------------------------------------------------------------
NUMERICAL FEATURES
--------------------------------------------------------------------------------
Feature              Min        Max        Mean       Median    
--------------------------------------------------------------------------------
age                  17.00      24.00      20.47      20.00     
sleep_hours          4.10       9.90       7.01       7.00      
class_attendance     40.60      99.40      70.02      69.90     
study_hours          0.08       7.91       4.01       4.04      
exam_score           19.60      100.00     62.51      62.60     

--------------------------------------------------------------------------------
CATEGORICAL FEATURES
--------------------------------------------------------------------------------

gender:
  Unique values: ['female', 'male', 'other']
  Value counts:
    other: 6726 (33.6%)
    male: 6695 (33.5%)
    female: 6579 (32.9%)

internet_access:
  U

In [ ]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(f"{base_folder}/data/housing.db")
housing = pd.read_sql_query(
    """
    SELECT
        b.block_id,
        b.longitude,
        b.latitude,
        s.housing_median_age,
        s.total_rooms,
        s.total_bedrooms,
        s.population,
        s.households,
        s.median_income,
        s.median_house_value,
        op.name AS ocean_proximity
    FROM block AS b
    JOIN block_housing_stats AS s
        ON s.block_id = b.block_id
    JOIN ocean_proximity AS op
        ON op.ocean_proximity_id = b.ocean_proximity_id
    ORDER BY b.block_id
    """,
    conn,
)
conn.close()

housing.head()

,block_id,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,-122.23,37.88,41.0,880,129.0,322,126,8.3252,452600.0,NEAR BAY
1,1,-122.22,37.86,21.0,7099,1106.0,2401,1138,8.3014,358500.0,NEAR BAY
2,2,-122.24,37.85,52.0,1467,190.0,496,177,7.2574,352100.0,NEAR BAY
3,3,-122.25,37.85,52.0,1274,235.0,558,219,5.6431,341300.0,NEAR BAY
4,4,-122.25,37.85,52.0,1627,280.0,565,259,3.8462,342200.0,NEAR BAY


In [ ]:
# =============================================================================
# ANALYZE HOUSING DATA FOR STREAMLIT APP
# Find min/max/median for numerical features and unique values for categorical features
# =============================================================================

import json

print("=" * 80)
print("ANALYZING HOUSING DATA FOR STREAMLIT APP")
print("=" * 80)

# Define the features we need for prediction
numerical_features = [
    'longitude',
    'latitude',
    'housing_median_age',
    'total_rooms',
    'total_bedrooms',
    'population',
    'households',
    'median_income'
]

categorical_features = ['ocean_proximity']

# Create schema dictionary
data_schema = {
    "numerical": {},
    "categorical": {}
}

# Analyze numerical features
print("\n" + "-" * 80)
print("NUMERICAL FEATURES")
print("-" * 80)
print(f"{'Feature':<25} {'Min':<15} {'Max':<15} {'Mean':<15} {'Median':<15}")
print("-" * 80)

for feature in numerical_features:
    min_val = float(housing[feature].min())
    max_val = float(housing[feature].max())
    mean_val = float(housing[feature].mean())
    median_val = float(housing[feature].median())

    data_schema["numerical"][feature] = {
        "min": min_val,
        "max": max_val,
        "mean": mean_val,
        "median": median_val
    }

    print(f"{feature:<25} {min_val:<15.2f} {max_val:<15.2f} {mean_val:<15.2f} {median_val:<15.2f}")

# Analyze categorical features
print("\n" + "-" * 80)
print("CATEGORICAL FEATURES")
print("-" * 80)

for feature in categorical_features:
    unique_values = housing[feature].unique().tolist()
    value_counts = housing[feature].value_counts().to_dict()

    data_schema["categorical"][feature] = {
        "unique_values": unique_values,
        "value_counts": value_counts
    }

    print(f"\n{feature}:")
    print(f"  Unique values: {unique_values}")
    print(f"  Value counts:")
    for value, count in value_counts.items():
        print(f"    {value}: {count} ({count/len(housing)*100:.1f}%)")

# Save schema to JSON file
output_file = f"{base_folder}/data/data_schema.json"
with open(output_file, 'w') as f:
    json.dump(data_schema, f, indent=2)

print("\n" + "=" * 80)
print(f"✓ Data schema saved to {output_file}")
print("=" * 80)

# Display the JSON structure
print("\n" + "-" * 80)
print("GENERATED SCHEMA (data_schema.json)")
print("-" * 80)
print(json.dumps(data_schema, indent=2))

print("\n" + "=" * 80)
print("DONE! Use data_schema.json in your Streamlit app")
print("=" * 80)

ANALYZING HOUSING DATA FOR STREAMLIT APP

--------------------------------------------------------------------------------
NUMERICAL FEATURES
--------------------------------------------------------------------------------
Feature                   Min             Max             Mean            Median         
--------------------------------------------------------------------------------
longitude                 -124.35         -114.31         -119.57         -118.49        
latitude                  32.54           41.95           35.63           34.26          
housing_median_age        1.00            52.00           28.64           29.00          
total_rooms               2.00            39320.00        2635.76         2127.00        
total_bedrooms            1.00            6445.00         537.87          435.00         
population                3.00            35682.00        1425.48         1166.00        
households                1.00            6082.00         499.54  